In [ ]:
import ipywidgets as widgets
import json
from IPython.display import display

# Sample JSON data
data = {
    "name": "John Doe",
    "age": 30,
    "city": "New York",
    "children": [
        {"name": "Jane", "age": 10},
        {"name": "Doe", "age": 8}
    ]
}

# Create an Output widget
output = widgets.Output()

# Function to update the JSON display
def update_json_display(change):
    data['age'] = age_input.value
    with output:
        output.clear_output()
        formatted_json = json.dumps(data, indent=4)
        print(formatted_json)

# Create an IntText widget for age input
age_input = widgets.IntText(
    value=data['age'],
    description='Age:',
    disabled=False
)

# Attach the update function to the IntText widget
age_input.observe(update_json_display, names='value')

# Display the initial JSON
with output:
    formatted_json = json.dumps(data, indent=4)
    print(formatted_json)

# Create a VBox to hold the input and output widgets
vbox = widgets.VBox([age_input, output])

# Display the VBox
display(vbox)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from timsy_notebook_utils import SqlUtil, sql_compiled_query_to_dataframe, sql_read, get_sql_scripts
from sqlalchemy import text
from project_manager import get_sql_scripts

engine = SqlUtil.engine()

def parse_query_parameters(input_query: str):
    query = text(input_query)
    compiled_pre_query = query.compile(engine)
    parameters = compiled_pre_query.params
    return parameters

def update_parameters():
    for param, widget in input_widgets.items():
        parameters[param] = widget.value

# Function to execute the query and display the results
def execute_query(e):
    update_parameters()
    query = text(input_query)
    bound_query = query.bindparams(**parameters)
    compiled_query = bound_query.compile(engine, compile_kwargs={"literal_binds": True})
    df = sql_compiled_query_to_dataframe(compiled_query)
    output.clear_output()
    with output:
        display(df)

def update_form(change):
    global input_query, parameters, input_widgets
    input_query = sql_read(change['new'])
    parameters = parse_query_parameters(input_query)
    input_widgets = {param: widgets.Text(description=param) for param in parameters.keys()}
    input_form.children = list(input_widgets.values()) + [execute_button, output]

# Get the list of SQL scripts
sql_scripts = get_sql_scripts()

# Create a dropdown widget for selecting the SQL script
dropdown = widgets.Dropdown(
    options=sql_scripts,
    description='SQL Script:',
    # value=sql_scripts[0]  # Default to the first item
)

# Attach the update_form function to the dropdown's value change event
dropdown.observe(update_form, names='value')

# Create input widgets and other UI elements
input_widgets = {}
parameters = {}
input_query = sql_read(dropdown.value)
parameters = parse_query_parameters(input_query)
input_widgets = {param: widgets.Text(description=param) for param in parameters.keys()}

# Create a button to execute the query
execute_button = widgets.Button(description="Execute Query")
execute_button.on_click(execute_query)

# Create an output widget to display the results
output = widgets.Output()

# Create a VBox to hold the input widgets, execute button, and output widget
input_form = widgets.VBox(list(input_widgets.values()) + [execute_button, output])

# Display the dropdown and the form
display(dropdown, input_form)